### Take all vars and do boost

##### here we do preprocessing with filling in Nan values with medians or averages

In [1]:
import pandas as pd
pd.options.display.max_columns=500
import numpy as np
import os, pickle
import matplotlib.pyplot as plt
import seaborn as sns

# import scipy.stats as stats

from sklearn import ensemble, model_selection, metrics, datasets, tree, linear_model
import xgboost as xgb

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

from scipy.stats import spearmanr
from scipy.cluster import hierarchy

from sklearn.impute import SimpleImputer

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

## Work with Russia ( 643 )

In [2]:
with open("only_rus.pck", "br") as f:
    df_r=pickle.load(f)

In [3]:
df1=df_r.drop(['version','doi','A_WAVE','A_STUDY','B_COUNTRY','B_COUNTRY_ALPHA',
          'C_COW_NUM','C_COW_ALPHA','A_YEAR','D_INTERVIEW','J_INTDATE','FW_END',
          'FW_START','K_TIME_START','K_TIME_END','K_DURATION','Q_MODE','N_REGION_ISO',
          'N_REGION_WVS','N_TOWN','G_TOWNSIZE','H_SETTLEMENT','I_PSU','O1_LONGITUDE',
        'O2_LATITUDE','S_INTLANGUAGE','LNGE_ISO','E_RESPINT','F_INTPRIVACY','W_WEIGHT',
        'S018','Q33_3','Q34_3','Q35_3','Q82','Q82_EU','Q82_APEC','Q82_ARABLEAGUE','Q82_ASEAN',
        'Q82_AFRICANUNION','Q82_CIS','Q82_ECO','Q82_GULFCOOP','Q82_ISLCOOP','Q82_MERCOSUR',
        'Q82_NAFTA','Q82_OAS','Q82_SAARC','Q82_SCO','Q82_TLC','Q82_UNDP','Q91','Q92','Q93',
        'Q223','Q223_ABREV','Q223_LOCAL','Partyname','Partyabb','CPARTY','CPARTYABB',
        'v2psprlnks','v2xnp_client','v2xps_party','v2psbars','v2psorgs','v2psprbrch',
        'v2psplats','WVS_LibCon_PartyVoter','WVS_Polmistrust_PartyVoter','WVS_LR_MedianVoter',
        'GPS_V6_Scale','GPS_V8_Scale','GPS_V9','GPS_V10',
        'GPS_V11','GPS_V12','GPS_V13','GPS_V14',
        'GPS_V15','GPS_V16','GPS_V17','WVS_LR_PartyVoter','WVS_LibCon_MedianVoter',
        'ID_GPS','ID_PartyFacts','Type_Values','Type_Populism','Type_Populist_Values',
        'Type_Partysize_vote','Type_Partysize_seat','GPS_V4_Scale','I_AUTHORITY',
                #точно убираем - одно-два значения 
        'lifeexpectHDI','homiciderate','Refugeesorigin','internetusers','mobphone','fhregion',
        'polregfh','freestfh','prfhrat','prfhscore','clfhrat','clfhscore','democ',
        'autoc','polity','durable','regtype','ruleoflaw','corrupttransp','electintegr',
        'btiregion','btistatus','btidemstatus','btistate','btipolpart','btiruleoflaw','btistability',
        'btiintegration','btimarket','btigovindex',
        'btigoveperform','btiregime','regionWB','incomeWB','landWB','GDPpercap1','GDPpercap2',
        'giniWB','incrichest10p','popWB1990','popWB2000','popWB2019','lifeexpect','popgrowth',
        'urbanpop','laborforce','deathrate','unemployfem','unemploymale','unemploytotal','accessclfuel',
        'accesselectr','renewelectr','co2emis','co2percap','easeofbusiness','militaryexp','Trade',
        'healthexp','educationexp','medageun','meanschooling','educationHDI','compulseduc','GII',
        'DGI','womenparl','hdi','incomeindexHDI','humanineqiality','migrationrate','schoolgpi',
        'femchoutsch','choutsch','v2x_polyarchy','v2x_libdem','v2x_partipdem','v2x_delibdem',
        'v2x_egaldem','v2x_freexp_altinf','v2x_frassoc_thick','v2xel_frefair','v2xcl_rol',
        'v2x_cspart','v2xeg_eqdr','v2excrptps','v2exthftps','v2juaccnt','v2cltrnslw','v2clacjust',
        'v2clsocgrp','v2clacfree','v2clrelig','v2csrlgrep','v2mecenefm','v2mecenefi','v2mebias',
        'v2pepwrses','v2pepwrgen','v2peedueq','v2pehealth','v2peapsecon','v2peasjsoecon',
        'v2clgencl','v2peasjgen','v2peasbgen','v2cafres','v2cafexch','v2x_corr','v2x_gender',
        'v2x_gencl','v2x_genpp','v2x_rule','v2xcl_acjst','PWGHT','E1_LITERACY','S025',
        'I_NATIONALISM','I_DEVOUT','I_TRUSTARMY','I_TRUSTPOLICE','I_TRUSTCOURTS',
        'I_RELIGIMP','I_RELIGBEL','I_RELIGPRAC','I_NORM1','I_NORM2','I_NORM3','RELATIVISM',
        'SACSECVAL','RESEMAVAL','DEFIANCE','DISBELIEF','SCEPTICISM',
        'I_INDEP','I_IMAGIN','I_NONOBED','AUTONOMY','I_WOMJOB','I_WOMPOL','I_WOMEDU','EQUALITY','I_HOMOLIB','I_ABORTLIB','Y001','Y002','Y003',
        'I_DIVORLIB','CHOICE','I_VOICE1','I_VOICE2','I_VOI2_00','VOICE','SECVALWGT','RESEMAVALWGT',
        'Q68', #профсоюз labour union
        'Q275R','Q275A',  #highest educ other scales  [possible to leave R]
        'Q276R','Q276A',  #highest educ other scales for spouze
        'Q277R','Q277A',  #highest educ other scales for mother
        'Q278R','Q278A',  #highest educ other scales for father
        'Q288R', #scale of incomes
        'Q73', #доверие парламенту
        'Q76','Q77',
#         'Q167', #believe in hell
               
        'Q79','Q80','Q81','Q83','Q84','Q85','Q86','Q87','Q88','Q89','Q90','Q289CS9',
        'Q112','Q113','Q114','Q115','Q116', #взятки
        'Q117', #журналисты взятки
        'Q118', #дают взятки
#         'Q136', #drugs in neighb
        'Q209', #petition
        'Q221','Q222','Q224','Q226','Q228','Q232','Q233','Q234','Q235',
        'Q225', #opposition is prevented from running
        'Q227', #voters are brided
        'Q229','Q230','Q231','Q234A','Q236',
        'Q237','Q238','Q239','Q240', #political
        'X003R2','x003r',
        'Q261', #год рождения
#         'Q284', #Sector of employment - no tendency
#         'Q285', #possibly NOT reliable - Are you the chief wage earner in your house
#         'Q289', #orthodox(~490), muslim,,,
#         'Q290' #Челябинск, краснодарская, дагест...
              ],axis=1)

df2 = df1[(df1['Q260'] == 2) & (df1['Q262'] > 39)].drop(['Q260','Q262'],axis=1)
df2.reset_index(inplace=True, drop=True)

#Transform kids number
df2['Q274ed'] = np.where(df2['Q274']>4, 4, df2['Q274'])
df2['Q274ed_cl'] = np.where(df2['Q274']>0, 1, 0)
df2.drop(['Q274'],axis=1, inplace = True)

#all negatives are nans
df2[df2<0] = np.nan

#fill in nans
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer = imputer.fit(df2)
df3 = imputer.transform(df2)
df3 = pd.DataFrame(data = df3, columns= df2.columns.values)

In [4]:
# df2['Q274'].isna().sum()

## Transform columns that have only two responses

In [5]:
col_two_responses = []
for col in df3.columns:
    if len(df3[col].value_counts()) == 2:
        col_two_responses.append(col)
#         print(col)
#         df3[col] = np.where(df3[col] == 2, 0, df3[col])
col_two_responses.pop()
print(col_two_responses)
df3_one_hot = pd.get_dummies(df3, prefix=col_two_responses, columns=col_two_responses)


['H_URBRURAL', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q57', 'Q139', 'Q140', 'Q141', 'Q144', 'Q145', 'Q149', 'Q150', 'Q151', 'Q165', 'Q166', 'Q167', 'Q168', 'Q174', 'Q175', 'Q263', 'Q264', 'Q265', 'Q269', 'Q285']


## Value counts and crosstab

In [6]:
# df3['Q289'].value_counts()

In [7]:
tmp1 = pd.crosstab(df3['Q274ed'], df3['Q283'], normalize=True) * 100
tmp1

Q283,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
Q274ed,,,,,,,,,,,
0.0,0.149701,1.347305,0.149701,0.000000,0.299401,0.748503,1.796407,3.742515,0.149701,0.299401,0.000000
1.0,0.299401,3.742515,1.347305,0.748503,0.449102,1.497006,5.538922,17.365269,2.694611,5.538922,0.449102
2.0,0.149701,2.245509,1.347305,0.449102,0.299401,1.646707,4.341317,16.916168,2.844311,7.035928,0.299401
3.0,0.449102,0.000000,0.449102,0.149701,0.299401,0.449102,1.347305,3.443114,1.197605,2.395210,0.000000
4.0,0.000000,0.149701,0.000000,0.000000,0.149701,0.000000,0.299401,1.646707,0.598802,0.898204,0.149701


In [ ]:
tmp2 = pd.DataFrame(tmp1.values, columns = ['0', '1'])

In [ ]:
tmp2['%'] = tmp2['1']/tmp2['0']

In [ ]:
tmp2

In [ ]:
# df3=df3[['Q210','Q279','Q273','Q113','Q89','Q270','Q94','Q248','Q278','Q172','Q261','Q183','Q56','Q274']]

## What are columns X003R2 x003r  ???

In [ ]:
# plt.hist(df2[df2['Q260']==2]['Q262'],bins=100);

In [ ]:
# for i,el in enumerate(df3.dtypes.values):
#     if not(el == 'int64' or el == 'float64'):
#         print(i, el)
# for col in df3.columns.values:
#     if df3[col].dtype not in ['int64','float64']: 
#         print(col, df3[col].dtype)

In [ ]:
# len(df3[~df3.isin([np.inf, -np.inf, -4]).any(1)])
# df_total = df_total.reset_index(drop=True)


**correlation matrix**


In [ ]:
corr_m = df3.corr()

In [ ]:
corr_m

In [ ]:
corr_m[(corr_m > -0.3) & (corr_m < 0.3)]

In [ ]:
plt.figure(figsize=(16, 16))
sns.heatmap(corr_m, 
        xticklabels=corr_m.columns,
        yticklabels=corr_m.columns,cmap="Greens");

In [ ]:
# col='PWGHT'
plt.hist(df3[col], bins=100);

In [ ]:
print(df3[col].min())
print(df3[col].max())
print(np.round(df3[col],1).value_counts)

In [ ]:
# for col in df3.columns:
#     print(col, df3[col].isna().sum())

In [ ]:
X.isna().sum().sum()

In [ ]:
# !nvidia-smi

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# X_pca = pca.transform(X)

In [ ]:
expl_var = pca.explained_variance_ratio_
print(expl_var[:250].sum())

In [ ]:
plt.figure(figsize=(16,6))
plt.bar(range(70), pca.explained_variance_ratio_[:70], label="df")
plt.title("Explained variance ratio for original df")
# plt.bar(range(50), expl_var[:50], label="df")
# plt.ylim([0,0.1])
plt.legend()
plt.show()

## Classifier

In [8]:
df3.columns.values

array(['G_TOWNSIZE2', 'H_URBRURAL', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6',
       'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16',
       'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25',
       'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34',
       'Q35', 'Q36', 'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43',
       'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52',
       'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60', 'Q61',
       'Q62', 'Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q69', 'Q70', 'Q71',
       'Q72', 'Q74', 'Q75', 'Q78', 'Q94', 'Q95', 'Q96', 'Q97', 'Q98',
       'Q99', 'Q100', 'Q101', 'Q102', 'Q103', 'Q104', 'Q105', 'Q106',
       'Q107', 'Q108', 'Q109', 'Q110', 'Q111', 'Q119', 'Q120', 'Q121',
       'Q122', 'Q123', 'Q124', 'Q125', 'Q126', 'Q127', 'Q128', 'Q129',
       'Q130', 'Q131', 'Q132', 'Q133', 'Q134', 'Q135', 'Q136', 'Q137',
       'Q138', 'Q139', 'Q140', 'Q141', 'Q142', 'Q143', 'Q144', 'Q145',
       'Q1

In [9]:
GPUn=3
for _ in range(7):
    df4 = df3_one_hot.sample(frac = 1)
    # df4.columns.values
    X = df4.drop(columns=['Q274ed','Q274ed_cl'])
    y = df4['Q274ed_cl']
    y = y.astype(int)

    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                    test_size = 0.20, shuffle=False)
    eval_set = [(X_train, y_train), (X_test, y_test)] #errors log container
    bst = xgb.XGBClassifier(objective ='binary:logistic', 
                       max_depth=5, 
                       n_estimators=1000, #1000 
                       eta = 0.05,
                       random_state = 40,
                       tree_method='gpu_hist', 
                        use_label_encoder=False,
                       gpu_id=GPUn)
    bst.fit(X_train, y_train, eval_set=eval_set, eval_metric= "rmse", verbose=False)
    y_pred = bst.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, pos_label=1, average='binary')
    print(_, accuracy, f1)
    mdi = bst.feature_importances_
    mdi_idx=np.argsort(mdi)[::-1]
    for i in mdi_idx[:7]:
        print(i, X_train.columns.values[i], mdi[i])
    print('~~~~~~~~~~~~~~~~~')

0 0.8880597014925373 0.9402390438247011
30 Q50 0.021812823
109 Q164 0.02042409
96 Q148 0.019852664
143 Q204 0.01752202
256 Q151_2.0 0.017431008
16 Q36 0.017347518
79 Q126 0.017105583
~~~~~~~~~~~~~~~~~
1 0.8805970149253731 0.9365079365079365
139 Q200 0.027567582
72 Q119 0.022148525
257 Q165_1.0 0.020369332
49 Q71 0.01742167
130 Q191 0.016471894
94 Q146 0.016207289
184 Q273 0.015958726
~~~~~~~~~~~~~~~~~
2 0.8955223880597015 0.944
96 Q148 0.04172879
141 Q202 0.02420201
72 Q119 0.023247976
139 Q200 0.020752555
26 Q46 0.019739931
16 Q36 0.017176857
43 Q64 0.017154602
~~~~~~~~~~~~~~~~~
3 0.9029850746268657 0.9486166007905139
126 Q187 0.027694402
96 Q148 0.023194384
26 Q46 0.022391684
36 Q56 0.022049157
43 Q64 0.017910102
128 Q189 0.017239379
0 G_TOWNSIZE2 0.016326644
~~~~~~~~~~~~~~~~~
4 0.8955223880597015 0.9448818897637796
139 Q200 0.02571239
21 Q41 0.024122106
72 Q119 0.02372087
143 Q204 0.023030855
207 Q11_1.0 0.02247535
44 Q65 0.01943607
184 Q273 0.018028783
~~~~~~~~~~~~~~~~~
5 0.9029850

## Regression model

In [9]:
GPUn=3
for _ in range(7):
    df4 = df3.sample(frac = 1)
    X = df4.drop(columns=['Q274ed','Q274ed_cl'])
    y = df4['Q274ed']
    
#     sc=StandardScaler()
#     sc.fit(X)
#     X_sc = sc.transform(X)
#     pca = PCA(n_components=250)  #
#     pca.fit(X_sc)
#     X_sc_pca = pca.transform(X_sc)

#     r_state = np.random.randint(200)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                    test_size = 0.20,
                    shuffle=False)
    eval_set = [(X_train, y_train), (X_test, y_test)] #errors log container
    bst = xgb.XGBRegressor(objective ='reg:squarederror', 
                       max_depth=5, 
                       n_estimators=1000, #1000 
                       eta = 0.05,
                       random_state = 40,
                       tree_method='gpu_hist', 
                       gpu_id=GPUn)
    bst.fit(X_train, y_train, eval_set=eval_set, eval_metric= "rmse", verbose=False)
    y_pred = bst.predict(X_test)
    r2_out = metrics.r2_score(y_test, y_pred)
    print(_, r2_out)


0 0.07080600946488746
1 0.05666642704501257
2 0.01933084970801302
3 0.0394626321536653
4 -0.06436695246328861
5 0.14858016850805733
6 0.03114696713095444


In [ ]:
0 -0.006763643580687484
1 0.14132652561194048
2 -0.013385769902816147
3 0.051214758273871674
4 0.12107522389                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  b                                                           bh87h6t6ewb6cr5rccf                                                                                                                                                                                                                                                    790892
5 0.08616023974553089
6 0.041869748306545884

In [ ]:
r2_out

In [ ]:
# for col in X.columns.values:
#     print(np.isfinite(X[col]).sum()-668)


### Dendrogramm

In [ ]:

def dual_plot(x_train):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,12))
    corr = spearmanr(x_train).correlation
    corr_linkage = hierarchy.ward(corr)

    dendro = hierarchy.dendrogram(corr_linkage, labels=list(x_train.columns), ax=ax1, leaf_rotation=90)
    dendro_idx = np.arange(0, len(dendro['ivl']))

    ax2.imshow(corr[dendro['leaves'], :][:, dendro['leaves']])
    ax2.set_xticks(dendro_idx)
    ax2.set_yticks(dendro_idx)
    ax2.set_xticklabels(dendro['ivl'], rotation='vertical')
    ax2.set_yticklabels(dendro['ivl'])
    fig.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
dual_plot(X)

### MDA

In [ ]:
from sklearn.inspection import permutation_importance 
from mlfinlab.clustering.feature_clusters import get_feature_clusters
result = permutation_importance(bst, X_test, y_test, n_repeats=4, random_state=42, n_jobs=2) #важность скорингом

In [ ]:
mda_perm_sorted_idx = result.importances_mean.argsort() #усреднение и сортировка

In [ ]:
c = 0
d = dict()
for el, res in zip(X.columns.values, result.importances_mean):
    print(c, el, res)
    d[el] = res
    c+=1

In [ ]:
feature_clusters = get_feature_clusters(X, dependence_metric='linear', n_clusters=13, distance_metric='angular', linkage_method='ward')  #add one to n_clusters

In [ ]:
feature_clusters

In [ ]:
for group in feature_clusters:
    max_res = dict()
    for el in group:
        max_res[el] = d[el]
    print(max_res)
    print(max(max_res, key=max_res.get))
    print('---------------')

In [ ]:
feature_clusters

In [ ]:
mda_perm_sorted_idx

In [ ]:
tree_indices = np.arange(0, N_feat ) + 0.5

In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))

ax1.barh(tree_indices, feature_scores_mdi[mdi_perm_sorted_idx[-N_feat:]], height=0.7)
ax1.set_yticklabels(feature_names[mdi_perm_sorted_idx[-N_feat:]])
ax1.set_yticks(tree_indices)
ax1.set_ylim((0, N_feat))
ax1.set_title('MDI')

ax2.boxplot(result.importances[mda_perm_sorted_idx[-N_feat:]].T, vert=False,
            labels=np.array(feature_names)[mda_perm_sorted_idx[-N_feat:]])
ax2.set_title('MDA')
fig.tight_layout()
plt.show()

In [ ]:

    
print("Russia length: ",len(df_r))
# take females and over 40 years
df_nec = df_r[(df_r['Q260'] == 2) & (df_r['Q262'] > 39)].copy()
print("Women over 40 years in Russia: ", len(df_nec))

# "H_URBRURAL","Q1", "Q3", "Q5","Q18","Q19","Q20","Q21","Q22","Q23","Q24","Q33","Q47","Q58","Q67","Q69",
# "Q108","Q109","Q138","Q161","Q172","Q206","Q213","Q238","Q252","Q262","Q263","Q271","Q273","Q275R",
# "Q279","Q288","Q274"
df_col = df_nec[["H_URBRURAL","Q1","Q3","Q5","Q18","Q19","Q20","Q21","Q22","Q23","Q24","Q33",
                 "Q47","Q58","Q67","Q69","Q108","Q109","Q138","Q161","Q172","Q206","Q213",
                 "Q238","Q252","Q263","Q271","Q273","Q275R","Q279","Q288","Q274",
                 "Q260","Q262"]].copy()
# df_col.Q274.unique()
df_col[df_col < 0] = np.nan
df_col["Q274"].replace({9: np.nan}, inplace=True)
df_col.dropna(inplace=True)
print("With dropped NaNs length is: ", len(df_col))
# df_col.columns.values

In [ ]:
#drop sex and age columns
df_col.drop(["Q260", "Q262"],axis=1, inplace=True)

In [ ]:
# df_nec.isna().sum()

In [ ]:
df_col.loc[df_nec["H_URBRURAL"] == 2, "H_URBRURAL"] = 0
df_col.loc[df_nec["Q18"] == 2, "Q18"] = 0
df_col.loc[df_nec["Q19"] == 2, "Q19"] = 0
df_col.loc[df_nec["Q20"] == 2, "Q20"] = 0
df_col.loc[df_nec["Q21"] == 2, "Q21"] = 0
df_col.loc[df_nec["Q22"] == 2, "Q22"] = 0
df_col.loc[df_nec["Q23"] == 2, "Q23"] = 0
df_col.loc[df_nec["Q24"] == 2, "Q24"] = 0
df_col.loc[df_nec["Q263"] == 2, "Q263"] = 0

In [ ]:
#ed = edited
df_col["Q271ed"] = np.where(df_col["Q271"]<2,0,1)
df_col["Q273ed"] = np.where(df_col["Q273"]<3,1,0)
df_col["Q279ed"] = np.where(df_col["Q279"]<4,1,0)
df_col.drop(["Q271","Q273","Q279"], axis=1, inplace=True)

In [ ]:
# plt.hist(df_col["Q33"], bins=100);

In [ ]:
# Number of features

# a=["H_URBRURAL","Q1", "Q3", "Q5","Q18","Q20","Q21","Q22","Q23","Q24","Q33","Q47","Q58","Q67","Q69",
# "Q108","Q109","Q138","Q161","Q172","Q206","Q213","Q238","Q252","Q263","Q271","Q273","Q275R",
# "Q279","Q288","Q274"]
# len(a)

#### all necessary features
"H_URBRURAL", 1.- Urban, 2.- Rural [2 -> 0]
"Q1", 1.- Very important, 4.- Not at all important
"Q3", 1.- Very important, 4.- Not at all important
"Q5", 1.- Very important, 4.- Not at all important
"Q18", 1.- Mentioned, 2.- Rural [2 -> 0]
"Q19", 1.- Mentioned, 2.- Rural [2 -> 0]
"Q20",1.- Mentioned, 2.- Rural [2 -> 0]
"Q21", 1.- Mentioned, 2.- Rural [2 -> 0]
"Q22",1.- Mentioned, 2.- Rural [2 -> 0]
"Q23", 1.- Mentioned, 2.- Rural [2 -> 0]
"Q24", 1.- Mentioned, 2.- Rural [2 -> 0]
"Q33", 1.- Agree strongly 5.- Disagree strongly
"Q47", 1.- Very good, 5.- Very poor
"Q58", 1.- Trust completely 4.- Do not trust at all
"Q67", 1.- A great deal 4.- None at all
"Q69", 1.- A great deal 4.- None at all
"Q108", 1.- governments responsibility, 10.- People responsibility 
"Q109", 1.- Competition is good, 10.- Competition is harmful 

"Q138", 1.- Very Frequently, 4.- Not at all frequently 

"Q161", 1.- Completely disagree, 10-comletely agree
"Q172", 1-often, 8-never
"Q206", 1.- Daily, 5.- Never
"Q213", 1.- Have done 2.- Might do 3.- Would never do
"Q238", 1.- Very good 2.- Fairly good 3.- Fairly bad 4.- Very bad
"Q252", 1 means not satisfied at all and 10 means completely satisfied
"Q263", 1.- I am born in this country, 2.-  [2 -> 0]
"Q271", (1 - no, 2,3,4 - yes) one-hot [1 -> 0]
"Q273",  Marital status (1,2 yes[1] ; 3,4,5,6 [0]) 
"Q275R", 1.- Primary 2.- Secondary 3.- Post-secondary
"Q279", (1,2,3 yes [1] ; 4,5,6,7, 8 no [0])
"Q288", 1 indicates the lowest income group and 10 the highest income group in your country
"Q274" number of kids

In [ ]:
feature_names  = X.columns.values        #имена всех фич
feature_scores_mdi = bst.feature_importances_  #важность фич

N_feat = len(feature_scores_mdi)               #количество фич
print(N_feat) 

In [ ]:
feature_scores_mdi

In [ ]:
mdi_perm_sorted_idx = feature_scores_mdi.argsort() #индексы для сортировки
mdi_perm_sorted_idx

### boost with "Q24","Q206","Q21","Q109","Q67","Q238","Q288","Q3","Q33"

In [ ]:
f = ["Q24","Q206","Q21","Q109","Q67","Q238","Q288","Q3","Q33"]

In [ ]:
X = df_col[f]
y = df_col['Q274']

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                test_size = 0.20, 
                random_state=17,
                shuffle=False)

In [ ]:
# !nvidia-smi

#### boost with "Q288,","H_URBRURAL","Q172","Q252","Q109","Q21","Q206","Q161","Q23","Q5","Q275R","Q108","Q3","Q67","Q263"


In [ ]:
len(df_col)

In [ ]:
# f = ["Q288","Q172","Q252","Q21","Q206","Q161","Q23","Q5"]
# f=['H_URBRURAL', 'Q1', 'Q3', 'Q5', 'Q18', 'Q19', 'Q21', 'Q22',
#        'Q23', 'Q24', 'Q33', 'Q47', 'Q58', 'Q67',  'Q108', 'Q109',
#        'Q161', 'Q206',  'Q263', 'Q275R', 'Q288',
#     'Q271ed', 'Q273ed', 'Q279ed']
f=["H_URBRURAL","Q288","Q161","Q109","Q206","Q108","Q21","Q67","Q33","Q5","Q47","Q22","Q24","Q271ed","Q1","Q3","Q19"]

# 'Q252','Q20','Q172','Q238', 'Q213', 'Q138','Q69',
X = df_col[f]
y = df_col['Q274']

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                test_size = 0.20, 
                random_state=26,
                shuffle=False)

In [ ]:
GPUn=1

eval_set = [(X_train, y_train), (X_test, y_test)] #errors log container
# eval_set = [(X, y)] #errors log container
bst = xgb.XGBRegressor(objective ='reg:squarederror', 
                   max_depth=5, 
                   n_estimators=1000, #1000 
                   eta = 0.05,
                   random_state = 21,
                   tree_method='gpu_hist', 
                   gpu_id=GPUn)
bst.fit(X_train, y_train, eval_set=eval_set, eval_metric= "rmse", verbose=False)
# bst.fit(X, y, eval_set=eval_set, eval_metric= "rmse", verbose=False)
y_pred = bst.predict(X_test)
# y_pred = bst.predict(X)
r2_out = metrics.r2_score(y_test, y_pred)
# sp_out = pd.DataFrame({"y_test":y_test, "y_pred":y_pred}).corr(method="spearman")["y_test"]["y_pred"] #spearman


In [ ]:
r2_out

In [ ]:
r = permutation_importance(bst, X_test, y_test, n_repeats=4, n_jobs=10, random_state=42)
mda_idx = r.importances_mean.argsort()

In [ ]:
N_feat=X.shape[1]
feature_names=X.columns.values
plt.figure(figsize=(14,18))
plt.boxplot(r.importances[mda_idx[-N_feat:]].T, vert=False,
            labels=np.array(feature_names)[mda_idx[-N_feat:]])
plt.title('MDA: replace features with random values and see how r2 decreases')
plt.show()

In [ ]:
explainer = shap.TreeExplainer(bst)
shap_values = explainer.shap_values(X_train)

# summarize the effects of all the features
shap.summary_plot(shap_values, X_train)

In [ ]:
H_URBRURAL, Q288 Q161 Q206 Q109 Q108 Q47 Q21 Q67 Q33 Q22 Q5  Q271ed Q24
    Q1 Q3  Q19


Q279ed   Q58

In [ ]:
feature_clusters

In [ ]:
r = permutation_importance(bst, X, y, n_repeats=4, n_jobs=10, random_state=42)
mda_idx = r.importances_mean.argsort()

In [ ]:
N_feat=X.shape[1]
feature_names=X.columns.values
plt.figure(figsize=(14,18))
plt.boxplot(r.importances[mda_idx[-N_feat:]].T, vert=False,
            labels=np.array(feature_names)[mda_idx[-N_feat:]])
plt.title('MDA: replace features with random values and see how r2 decreases')
plt.show()

In [ ]:
import shap
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(bst)
shap_values = explainer.shap_values(X)

# summarize the effects of all the features
shap.summary_plot(shap_values, X)

In [ ]:
#Pearson
np.corrcoef(y_test, y_pred)[0,1]

#### Dendrogramm

In [ ]:
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
def dual_plot(x):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,12))
    corr = spearmanr(x).correlation
    corr_linkage = hierarchy.ward(corr)

    dendro = hierarchy.dendrogram(corr_linkage, labels=list(x.columns), ax=ax1, leaf_rotation=90)
    dendro_idx = np.arange(0, len(dendro['ivl']))

    ax2.imshow(corr[dendro['leaves'], :][:, dendro['leaves']])
    ax2.set_xticks(dendro_idx)
    ax2.set_yticks(dendro_idx)
    ax2.set_xticklabels(dendro['ivl'], rotation='vertical')
    ax2.set_yticklabels(dendro['ivl'])
    fig.tight_layout()
    plt.show()

In [ ]:
dual_plot(X)

## Corr matrix and heatmap

In [ ]:
c = df_col_tmp["Q274"].corr(method='pearson')

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(c, 
        xticklabels=c.columns,
        yticklabels=c.columns,cmap="Greens");

In [ ]:
idx = np.argsort(df_col_tmp.corr()['Q274'])
df_col_tmp.corr()['Q274'][idx]

In [ ]:
# !nvidia-smi

#### Lin regr with selected by Spearman corr with Q274 features

In [ ]:
Q279ed         -0.164232
H_URBRURAL_1   -0.160339
Q172           -0.142834
Q275R_3.0      -0.138231
Q288           -0.080889
Q3              0.072417
Q262sq          0.074490
Q273ed          0.075786
Q263_2          0.076532
Q262ed          0.097694
Q275R_1.0       0.121706
H_URBRURAL_2


In [ ]:
# f = ["Q288","Q172","Q252","Q21","Q206","Q161","Q23","Q5","Q275R","Q67"]
# X = df_col[f]
# y = df_col['Q274']

df_selected = df_col[["Q274","Q172","Q275R","Q288","Q3","Q263","H_URBRURAL"]]
X = df_selected.drop(["Q274"], axis=1)
y = df_selected['Q274']


In [ ]:
X_train,X_test,y_train,y_test=model_selection.train_test_split(X, y,
        test_size = 0.30,
        random_state=42,
        shuffle=False)

In [ ]:
X2_train = sm.add_constant(X_train)
X2_test = sm.add_constant(X_test)
est = sm.OLS(y_train, X2_train)
est2 = est.fit()
print ("In-Sample: %s" % r2_score(y_train, est2.predict(X2_train)))
print ("Out-of-Sample: %s" % r2_score(y_test, est2.predict(X2_test)))
print("\n")
print(est2.summary())
   

In [ ]:
cols = list()
for i in range(177,196):
    cols.append('Q'+str(i))

In [ ]:
df_col = df_r[cols]

In [ ]:
df_col.head()

In [ ]:
# with open("only_rus_q177_195.pck", "bw") as f:
#     pickle.dump(df_col, f)

In [ ]:
# with open("only_rus_q177_195.pck", "br") as f:
#     df_col=pickle.load(f)


In [ ]:
len(df_col.Q179)

In [ ]:
df_col.Q179.value_counts()

In [ ]:
df1 = pd.DataFrame()
for col in df_col.columns:
    df1[col] = df_col[col].replace(-1,np.nan)
    df1[col] = df_col[col].replace(-2,np.nan)
    df1[col] = df_col[col].replace(-4,np.nan)
    df1[col] = df_col[col].replace(-5,np.nan)

In [ ]:
df1.isna().sum()

In [ ]:
df1.dropna(axis=1, inplace=True)

In [ ]:
len(df1)

In [ ]:
df1 = df1/10

In [ ]:
df1.

In [ ]:
from seaborn import distplot, pairplot

In [ ]:
pairplot(df1)

## Corr matrix and heatmap

In [ ]:
c = df1.corr(method='pearson')

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(c, 
        xticklabels=c.columns,
        yticklabels=c.columns,cmap="Greens");

## Define features and target

In [ ]:
X = df1.drop(columns=['Q186'])
y = df1['Q186']

## Split to train and test

In [ ]:
X_train,X_test,y_train,y_test=model_selection.train_test_split(X, y,
        test_size = 0.30,
        random_state=42,
        shuffle=True)

## Linear Regression

In [ ]:

X2_train = sm.add_constant(X_train)
X2_test = sm.add_constant(X_test)
est = sm.OLS(y_train, X2_train)
est2 = est.fit()
print ("In-Sample: %s" % r2_score(y_train, est2.predict(X2_train)))
print ("Out-of-Sample: %s" % r2_score(y_test, est2.predict(X2_test)))
print("\n")
print(est2.summary())
   

## XGBoost fit

In [ ]:
eval_set = [(X_train, y_train), (X_test, y_test)] #error logs container during training
bst = xgb.XGBRegressor(objective ='reg:squarederror',
                       max_depth=5,
                       n_estimators=200,
                       eta = 0.03,
                       tree_method='gpu_hist', 
                       gpu_id=3)
bst.fit(X_train, y_train, eval_set=eval_set, eval_metric= "rmse", verbose=False)
print ("Out-of-Sample: %s" % metrics.r2_score(y_test, bst.predict(X_test)))
print ("In-Sample: %s" % metrics.r2_score(y_train, bst.predict(X_train)))

In [ ]:
from xgboost import plot_tree
fig, ax = plt.subplots(figsize=(100, 100))
xgb.plot_tree(bst, num_trees=4, ax=ax)
plt.show()

## Clustering

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


In [ ]:
kmeans = KMeans(
   init="random",
   n_clusters=3,
   n_init=10,
   max_iter=300,
   random_state=42)

In [ ]:
kmeans.fit(df1)

In [ ]:
# The lowest SSE value
print(kmeans.inertia_)

# Final locations of the centroid
print(kmeans.cluster_centers_)

# The number of iterations required to converge
print(kmeans.n_iter_)
    

In [ ]:
kmeans.labels_[:5]

In [ ]:
kmeans_kwargs = {
"init": "random",
"n_init": 10,
"max_iter": 300,
"random_state": 42,
}

# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(df1)
    sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()
    

In [ ]:
# A list holds the silhouette coefficients for each k
silhouette_coefficients = []

# Notice you start at 2 clusters for silhouette coefficient
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(df1)
    score = silhouette_score(df1, kmeans.labels_)
    silhouette_coefficients.append(score)

In [ ]:
silhouette_coefficients

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
df_r_test = df_r[(df_r.Q1.isin([1,2,3,4])) & (df_r.Q2.isin([1,2,3,4]))][['Q1','Q2']]

In [ ]:
len(df_r_test)

In [ ]:
n = np.sum(df_r_test)

In [ ]:
n

In [ ]:
import scipy.stats as stats
X2 = stats.chi2_contingency(df_r_test, correction=False)[0]

In [ ]:
X2

In [ ]:
n = np.sum(df_r_test)
minDim = min(df_r_test.shape)-1

#calculate Cramer's V 
V = np.sqrt((X2/n) / minDim)

#display Cramer's V
print(V)

In [ ]:
df_r_test.groupby().count() 

In [ ]:
plt.hist(df_r.Q177, bins = 50);


In [ ]:
df_r.Q177.mean()

In [ ]:
df_r.Q177.std()

In [ ]:
len(df_r.Q177)

In [ ]:
df_r.replace(-5,np.nan,inplace=True)
df_r.replace(-4,np.nan,inplace=True)
df_r.replace(-2,np.nan,inplace=True)
df_r.replace(-1,np.nan,inplace=True)


# -1-.- Don´t know
# -2-.- No answer
# -4-.- Not asked
# -5-.- Missing; Not available